In [1]:
#pip install sentence-transformers #Installed package from https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2

In [ ]:
#pip install tf-keras

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [1]:
import numpy as np
import os
import random #I want the model to pick a random quote.
from google import genai
from google.genai import types
from sentence_transformers import SentenceTransformer #Importing from Hugging face so i can use a local model for embedding.
from pypdf import PdfReader
import nltk #Importing so i can get whole quotes (rows) even if there is . in the sentence.
#nltk.download("punkt")
#nltk.download("punkt_tab")
from nltk.tokenize import sent_tokenize
import json
from sklearn.metrics.pairwise import cosine_similarity #Importing for semantic searching.

client = genai.Client(api_key=os.getenv("API_KEY"))
model=SentenceTransformer('all-MiniLM-L6-v2') #Model used for local embedding.

/home/peter/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-05-24 10:49:02.387835: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748076542.402595    3791 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748076542.407009    3791 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1748076542.419661    3791 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:

In [2]:
response = client.models.generate_content( #Running to see if the model is working, and can leaave me a response to my question.
model="gemini-2.0-flash",
contents="Hi, who are you?"
)

print(response.text)

I am a large language model, trained by Google.



In [29]:
reader_stat = PdfReader("pdf/islr.pdf") #islr.pdf is the boook, An Introduction to Statistical Learning by Gareth James and more.
text_stat = ""
for page in reader_stat.pages:
    text_stat += page.extract_text()

reader_homl = PdfReader("pdf/homl.pdf") #homl.pdf is the boook, Hands-on Machine Learning with Scikit-Learn, Keras & TensorFlow (ML/DL).
text_homl = ""
for page in reader_homl.pages:
    text_homl += page.extract_text()

reader_lotr = PdfReader("pdf/lotr.pdf") #lotr.pdf is a pdf with some famous Lord of the ring quotes.
text_lotr = ""
for page in reader_lotr.pages:
    text_lotr += page.extract_text()

Ignoring wrong pointing object 173 0 (offset 0)
Ignoring wrong pointing object 376 0 (offset 0)
Ignoring wrong pointing object 393 0 (offset 0)
Ignoring wrong pointing object 425 0 (offset 0)
Ignoring wrong pointing object 427 0 (offset 0)
Ignoring wrong pointing object 434 0 (offset 0)
Ignoring wrong pointing object 652 0 (offset 0)
Ignoring wrong pointing object 678 0 (offset 0)
Ignoring wrong pointing object 781 0 (offset 0)
Ignoring wrong pointing object 837 0 (offset 0)
Ignoring wrong pointing object 840 0 (offset 0)
Ignoring wrong pointing object 843 0 (offset 0)
Ignoring wrong pointing object 854 0 (offset 0)
Ignoring wrong pointing object 885 0 (offset 0)
Ignoring wrong pointing object 929 0 (offset 0)
Ignoring wrong pointing object 1050 0 (offset 0)
Ignoring wrong pointing object 1092 0 (offset 0)
Ignoring wrong pointing object 1125 0 (offset 0)
Ignoring wrong pointing object 1138 0 (offset 0)
Ignoring wrong pointing object 1140 0 (offset 0)
Ignoring wrong pointing object 1149

In [30]:
def semantic_chunking(text, max_tokens=100): #Creating funtion for Semantic chunking to use in next step.
    sentences = sent_tokenize(text)
    chunks = []
    current_chunk = ""
    
    for sentence in sentences:
        # Check if adding the sentence keeps the chunk within the token limit
        if len((current_chunk + " " + sentence).split()) <= max_tokens:
            current_chunk += " " + sentence
        else:
            chunks.append(current_chunk.strip())
            current_chunk = sentence
    if current_chunk:
        chunks.append(current_chunk.strip())
    
    return chunks

In [31]:
chunks_stat = semantic_chunking(text_stat, max_tokens=100)
chunks_homl = semantic_chunking(text_homl, max_tokens=100)

sentences_lotr = sent_tokenize(text_lotr) #Not using Semantic search because i only have short quotes in this data.
sentences_lotr = [s.strip() for s in sentences_lotr if len(s.strip()) > 30]

In [37]:
print("STAT chunks:", len(chunks_stat))
print("HOML chunks:", len(chunks_homl))
print("TOTAL chunks:", len(chunks_stat) + len(chunks_homl))


STAT chunks: 2530
HOML chunks: 3130
TOTAL chunks: 5660


In [32]:
valid_chunks_homl = [] #Had problem with creating embedding for homl, need to clean out bad/non string chunks.

for i, chunk in enumerate(chunks_homl):
    try:
        model.encode([chunk])  # testing one each turn
        valid_chunks_homl.append(chunk)
    except Exception as e:
        print(f" Skipping broken chunk in index {i} → {e}")


 Skipping broken chunk in index 491 → TextEncodeInput must be Union[TextInputSequence, Tuple[InputSequence, InputSequence]]
 Skipping broken chunk in index 902 → TextEncodeInput must be Union[TextInputSequence, Tuple[InputSequence, InputSequence]]
 Skipping broken chunk in index 965 → TextEncodeInput must be Union[TextInputSequence, Tuple[InputSequence, InputSequence]]
 Skipping broken chunk in index 968 → TextEncodeInput must be Union[TextInputSequence, Tuple[InputSequence, InputSequence]]
 Skipping broken chunk in index 995 → TextEncodeInput must be Union[TextInputSequence, Tuple[InputSequence, InputSequence]]
 Skipping broken chunk in index 1007 → TextEncodeInput must be Union[TextInputSequence, Tuple[InputSequence, InputSequence]]
 Skipping broken chunk in index 1296 → TextEncodeInput must be Union[TextInputSequence, Tuple[InputSequence, InputSequence]]
 Skipping broken chunk in index 2148 → TextEncodeInput must be Union[TextInputSequence, Tuple[InputSequence, InputSequence]]
 Skip

In [ ]:
embeddings_homl = model.encode(valid_chunks_homl, show_progress_bar=True) #Using the filtered/working chunks for homl.
embeddings_stat = model.encode(chunks_stat, show_progress_bar=True)
embeddings_lotr = model.encode(sentences_lotr, show_progress_bar=True)

Batches: 100%|██████████| 3/3 [00:00<00:00, 101.73it/s]


In [3]:
#Creating save function for easier saving.
def save(texts, embeddings, filename):
    with open(filename, "w") as f:
        json.dump([{"text": t, "embedding": e.tolist()} for t, e in zip(texts, embeddings)], f)


In [ ]:
#Savinig embeddings for later use.
save(chunks_stat, embeddings_stat, "embeddings/embeddings_stat.json")
save(valid_chunks_homl, embeddings_homl, "embeddings/embeddings_homl.json")
save(sentences_lotr, embeddings_lotr, "embeddings/embeddings_lotr.json")

In [ ]:
#Wrote this code so i can quickly load embeddings without running all code above again. Saves time if i close the project and need to come back.
with open("embeddings/embeddings_stat.json") as f:
    data_stat = json.load(f)
texts_stat = [d["text"] for d in data_stat]
embeddings_stat = np.array([d["embedding"] for d in data_stat])

with open("embeddings/embeddings_homl.json") as f:
    data_homl = json.load(f)
texts_homl = [d["text"] for d in data_homl]
embeddings_homl = np.array([d["embedding"] for d in data_homl])

with open("embeddings/embeddings_lotr.json") as f:
    data_lotr = json.load(f)
texts_lotr = [d["text"] for d in data_lotr]
embeddings_lotr = np.array([d["embedding"] for d in data_lotr])

In [ ]:
#Tagging text so i know where the output came frome if i want to use it later.
texts_stat_tagged = [f"[STAT] {text}" for text in texts_stat]
texts_homl_tagged = [f"[HOML] {text}" for text in texts_homl]

#Combining both stat and homl so the model can search in both.
texts_combined = texts_stat_tagged + texts_homl_tagged
embeddings_combined = np.vstack([embeddings_stat, embeddings_homl])

In [1]:
def semantic_search(query, texts, embeddings, model, k=5): #Perform semantic search to find the chunks most similar in meaning to the user's question

    query_emb = model.encode([query])
    similarity_scores = cosine_similarity(query_emb, embeddings)[0]
    top_indices = similarity_scores.argsort()[-k:][::-1]
    return [(texts[i], similarity_scores[i]) for i in top_indices]

In [ ]:
#This prompt defines how the model should behave and respond to questions
system_prompt = """You are a helpful assistant who explains statistical and machine learning concepts clearly.

You will receive a question along with some context. You must base your answer **only on the provided context**, and not on any external knowledge.

If the context does not contain enough information to answer the question, simply reply:  
**"I don't know."** Do not attempt to guess or invent details.

Please write your answer in clear, simple language, and structure it into well-formed paragraphs.

Use a slightly whimsical tone, as if you were Gandalf explaining it over a campfire.

---

Context:
{context}

Question:
{query}

To make it more entertaining, include a Lord of the Rings quote that fits the context. You may **tweak the quote slightly** to make it funnier or more related to the topic.

At the end of your answer, include the quote on its own line, followed by the name of the character who said it.
For example:
"There is always hope." – Aragorn

Quote:
{lotr_quote}" – {lotr_author}
"""


In [ ]:
def generate_user_prompt(query, texts, embeddings, model, k=5): # Generates a user prompt by retrieving the most relevant context chunks based on semantic similarity
    context_chunks = semantic_search(query, texts, embeddings, model, k)
    context = "\n\n".join([chunk for chunk, score in context_chunks])
    return f"Question: {query}\n\nContext:\n{context}"


In [ ]:
def extract_lotr_author(quote): #I want to present the author of the quote later on so thats why I extract it seperatly.
    if "–" in quote:
        return quote.split("–")[-1].strip()
    elif "-" in quote:
        return quote.split("-")[-1].strip()
    else:
        return "Unknown"

In [ ]:
# Generates a response from the Gemini model using the user question, selected context, and a LOTR quote
def generate_response(system_prompt, query, texts, embeddings, model, client, lotr_quotes, gemini_model="gemini-2.0-flash"): 
    lotr_quote = random.choice(lotr_quotes)
    user_prompt = generate_user_prompt(query, texts, embeddings, model)

    filled_prompt = system_prompt.format(
        context=user_prompt.split("Context:\n")[1],
        query=query,
        lotr_quote=lotr_quote,
        lotr_author = extract_lotr_author(lotr_quote)
    )

    response = client.models.generate_content(
    model=gemini_model,
    contents=filled_prompt
    )

    return response.text

In [ ]:
print(generate_response( #Wanted to see how the model answers.
    system_prompt,
    query="What is Normal Distribution?",
    texts=texts_combined,
    embeddings=embeddings_combined,
    model=model,
    client=client,
    lotr_quotes=texts_lotr
))


Halt, traveler! Gather 'round the fire, and let's speak of the Normal Distribution, a shape as familiar to statisticians as the Shire is to Hobbits.

Imagine a bell, a grand, symmetrical bell. That's the shape of a Normal Distribution, also known as a Gaussian distribution. It describes how the values of a feature, like the heights of Elves or the prices of Lembas bread, are spread out. Most values cluster around the average, the mean – that's the peak of our bell. Then, as you move away from the mean, the values become less common, tapering off on either side.

There's even a rule of thumb to remember, a bit like an Elven rhyme: "68-95-99.7". About 68% of the values fall within one standard deviation (σ) of the mean, 95% within two σ, and a whopping 99.7% within three σ. This "standard deviation" measures how spread out the values are. A small standard deviation means the bell is narrow and tall, most values are close to the mean. A large standard deviation means the bell is wide and 

<h3>Evaluation of the model</h3>

In [23]:
validation_data = [ #Creating questions with ideal answers so i can evalute the models performance.
{"question": "What is a common performance metric for regression problems?",
 "ideal_answer": "A typical performance measure for regression problems is the Root Mean Square Error (RMSE)."
},
{"question": "When we fit a linear regression model to a particular data set, many problems may occur. which is the most common?:",
 "ideal_answer": "Common problems in linear regression include non-linearity, heteroscedasticity, correlated errors, outliers, high-leverage points, and collinearity."
},
{"question": "How many horses are in Lord of the Rings?",
 "ideal_answer": "There is no relevant information about horses in the provided context."
},
]

In [ ]:
#created an evalution prompt so i can measure the models performance. This answers if i need to tweak it or not.
evaluation_system_prompt = """You are an intelligent evaluation system that grades the quality of an AI assistant's answer.

You will be given:
- A user question
- An ideal answer (the key facts that must be included)
- The assistant's generated answer (which may have a whimsical tone)

Your task is to evaluate the assistant's answer based on factual correctness and coverage of key ideas, **regardless of style**.

Score:
- 1.0 if the assistant includes the key information from the ideal answer, even if expressed creatively or with humor
- 0.5 if the answer is partially correct or misses important parts
- 0.0 if the answer is incorrect or irrelevant

Also include a brief motivation for your score.
"""


Testing the model on a questions that does exist.

In [22]:
query = validation_data[0]["question"]
response = generate_response(
    system_prompt=system_prompt,
    query=query,
    texts=texts_combined,
    embeddings=embeddings_combined,
    model=model,
    client=client,
    lotr_quotes=texts_lotr
)

evaluation_prompt = f"""Question: {query}
AI-assistent answer: {response}
Ideal Answer: {validation_data[0]['ideal_answer']}"""

evaluation_response = client.models.generate_content(
    model="gemini-2.0-flash",
    contents=evaluation_prompt,
    config=types.GenerateContentConfig(
        system_instruction=evaluation_system_prompt,
        temperature=0.0
    )
)

print(evaluation_response.text)

Score: 1.0

Motivation:The AI assistant correctly identifies Root Mean Square Error (RMSE) as a common performance metric for regression problems. It also mentions Mean Squared Error (MSE) which is closely related and also correct. The answer is creative, but contains the correct information.


Testing the model on a questions that does not exist.

In [25]:
validation_data[2]

{'question': 'How many horses are in Lord of the Rings?',
 'ideal_answer': 'There is no relevant information about horses in the provided context.'}

In [26]:
query = validation_data[2]["question"]
response = generate_response(
    system_prompt=system_prompt,
    query=query,
    texts=texts_combined,
    embeddings=embeddings_combined,
    model=model,
    client=client,
    lotr_quotes=texts_lotr
)

evaluation_prompt = f"""Question: {query}
AI-assistent answer: {response}
Ideal Answer: {validation_data[2]['ideal_answer']}"""

evaluation_response = client.models.generate_content(
    model="gemini-2.0-flash",
    contents=evaluation_prompt,
    config=types.GenerateContentConfig(
        system_instruction=evaluation_system_prompt,
        temperature=0.0
    )
)

print(evaluation_response.text)

Score: 1.0

Motivation:The AI assistant correctly states that it does not know the answer to the question. It also acknowledges the lack of relevant information. The whimsical tone does not detract from the accuracy of the response.


In [ ]:
#Saving model so i can use it later.
model.save("model/my_model")

<h3>Fördjupad och kritisk diskussion</h3>

Användningsområden:<br>
På ett företag där personer som arbetar inom området AI, ML och med statistik. Modellen kan hjälpa medarbetarna i deras dagliga verksamhet genom att fungera som ett stöd där man kan ställa frågor och få svar för att lösa olika uppgifter.

På en skola eller annan verksamhet som arbetar med utbildning där både lärare och elever skulle kunna nyttja modellen för sitt lärande. Allt från att lära nytt, reptera eller ställa frågor till modellen om sådant man vill lösa.

Möjligheter:<br>
Spara tid vilket också är relaterat till pengar. De anställda kanske genom den frigjorda tiden kan ägna sig åt andra saker eller överlag kan lägga mer tid på det dem önskar och har behov av. Botten kan ge fördjupad förståelse och personlig utveckling för användaren. Möjlighet till repetion. Eventullt erbjuda detta som tjänst som verksamheten kan ta betalt för genom att andra utanför verksamheten kan nyttja den, så som stundenter eller kunder.

Risker:<br>
Självklart finns olika risker genom att bygga och implementera en sådan modell och även användadet av den.
Hur hanterar man känslig information? Skall det vara en modell som endast förekommer lokalt till följd av känslig information eller skall man kunna nyttja den i molnet? Vem skall ha tillgång till modellen? Skall även andra inom verksamheten kunna nyttja modellen eller endast ett urval? Hur kan vi "låsa" modellen eller på annat sätt begränsa den så att den inte sprids? Vilken typ av frågor eller känslig information får användarna lov att mata in i modellen? Vem skall har rättigheter att kunna fortsätta träna modellen?
Hur håller vi modellen uppdaterad med rätt och korrekt information? Det finns många frågeställningar att fundera över när det kommer till risker med modellen, men då möjligheterna är många och fördelarna troligen gynnsamma är det definitivt värt att arbeta fram en plan för att försöka minimera riskerna så att man kan implementera modellen.

Lagring:<br>
Hur skall vi lagra modellen? Skall det vara i molnet eller skall det vara lokalt inom verksamheten? Som vi vet finns både fördelar och nackadelar med detta. Så som tillgänlighet och kostnader. Beroende på verksamhet och de ekonomiska resursena samt känsligheten (syftar på om datan är känslig för verksamheten) för modellen, styr vilken lösning som bäst lämpar sig för varje organisation och hur den lagras och implementeras. 

Kostnader:<br>
Kostnaderna ser olika ut för om man använder modellen lokalt eller molnbaserat och här behöver man som verksamhet också ta ställning till tillgänglighet, utveckling, säkerhet resurser med mera som får styra hur man väljer att arbeta med och implementera sin modell. Skall vman använda en molnbaserad modell för att ständigt ha tillgång till de senaste utvecklingarna? Fördelen är att man har har tillgång till ny teknik men en nackdel är att det kan fort kosta väldigt mycket pengar om man behöver skicka mycket förfrågningar över API. Använder man lokala lösningar så kan man komma undan mycket kostnader för API men behöver stället själv lägga mer kostnader på hårdvara och personal som till och från behöver se till att modellen uppdateras efter behov när ny teknik är tillgänglig. Även här blir det tydligt att de ekonomiska musklerna och tillgängligheten blir styrande. 

Förhållningssätt :<br>
De allra flesta verksamheter kräver att de anställda skall efterleva deras kultur och språk, grafiska image med mera. Det är också något vi behöver tänka på vad gäller modellen och hur den skall vara utformad och hur den svarar. Det kan därför vara viktigt att utforma modellen så att den svarar som vi önskar, använder ett korrekt språk, följer normer och attityder som gäller i verksamheten.

Copyright/rättigheter:<br>
För att kunna nyttja modellen i verksamheten är det också väldigt viktigt att man har rättigheter till att använda det material som modellen tränas på så att detta sedan kan användas legitimt i verksamheten. Detta för att undvika juridiska påföljder så som stämningar eller andra åberopande som kan skada verksamheten både ekonomiskt men också förtroendet.

Förbättringar:<br>
En vidare utveckling av modellen skulle vara att den också kan bearbeta och hantera bilder i datan. Detta för att kunna modellen skall kunna inkludera bilder i sina svar vilket bidrar till en ökad förståelse när detta kombineras med texten. Två andra idéer skulle dels kunna vara att den kan tränas att svara på flera språk vilket gör det enklare för användaren att ta till sig informationen. Dels att koppla till en berättande röst som läser upp svaret för användaren som den så önskar. Skulle man spinna vidare på den lite roligare modellen så skulle man med fördel kunna nyttja en röst som låter som Gandalf vilket hade bidragit med lite extra humor till svaret.

Reflektioner:<br>
I praktiken kan det gå relativt snabbt att skapa och sjösätta en ChatBot som vi kan nyttja inom verksamheten. Men som nämnt ovan finns det en hel del att förhålla sig till. Det kan därför vara bra att man innan man bygger en ChatBot har funderat åtminstone lite kring vissa av de nämnda områdena och förankrat detta med berörda och beslutsfattare. För även om det är roligt att sätta igång och bygga en modell så kan det bli så att den inte kan nyttjas om den inte uppfyller de krav som ställs. Skulle man redan byggt en modell som inte kan användas så har man troligen redan lagt ner både mycket tid och resurser vilka går förlorade och som hade kunna nyttjats til något annat. Samtidigt om det är så att man funderat kring och bygger en ChatBot som faktiskt används inom verksamheten så finns där fantastiska möjligheter för verksamheten genom nyttjandet av en sådan. Med tanke på den utveckling vi ser och hur Chatbotar blir allt vanligare är det troligt att de olika organisationerna och företagen ser att nyttan överväger riskerna vilket gör det till ett spännande område att fördjupa sig i.


<h3>Teoretiska frågor</h3>

1. AI, maskininlärning (ML) och djupinlärning (DL) är relaterade där AI är det breda området, ML är en underkategori av AI, och DL är en vidareutveckling av ML.

2. TensorFlow är ramverket som tränar modellen, och Keras är ett förenklat lager ovanpå som låter en bygga och träna modeller genom ett lättanvänt API.

3. Hyperparametrar styr hur modellen tränas. Parametrar är vad modellen lär sig under träningen.

4. Träningsdata används för att träna modellen, så att den kan optimera sina beräkningar för att få till bästa generalisering.

    Valideringsdata används för att utvärdera modellens/modellernas prestation vilket ofta används för att välja vilken modell vi fortsatt vill använda.

    Testdata används för att se hur väl modellen generaliserar på osedd data. Detta ger oss insikten hur bra modellen verkligen är på att generalisera på inte tidigare skådad data.

5. Steg 1:<br>
Först hämtas antal parametrar (features) från x_train och lagras i n_cols
En sequenial modell instansieras i nn_model
Ett första (dolt) lager med 100 neuroner skapas. Aktiveringsfunktion="relu". Input_shape anger hur många features som finns vilka hämtas från n_cols.
Vi adderar en dropout om 0.2 för att regularisera modellen.
Ett andra (dolt) lager om 50 neuroner skapas. Aktiveringsfunktion="Relu".
Sist skapar vi ett output lager med 1 neuron för att få ut ett värde. Aktiveringsfunktion="Sigmoid", vilket ger en sannolikhet (0 till 1).

    Steg 2:<br>
    Kompilering av modellen.
    optimizer='adam' - används för att optimera modellen under träningen för att minimera förlusten.
    loss binary_crossentropy = används vid klassificiering för att mäta true mot predicted.
    metrics=['accuracy'] - används för att mätra hur bra vår modell presterar.

    Steg 3:<br>
    EarlyStopping(patience=5) - används för att avbryta träningen om modellen inte förbättras. Om den inte presterar bättre under 5 epoker i rad, stoppas träningen.
    Modellen tränas genom fit. Vi tränar modellen genom indata x_train och y_train. 20% av träningsdatan används som valideringsdata för att mäta modellens prestanda under träning.
    100 epoker anger hur många "varv" modellen potentiellt kan träna för att hitta bästa optimering.
    epochs=100 - anger hur många epoker (antal försök) modellen har på sig att träna för att förbättra sitt resultat.
    Callback="Early stopping" anropar vår Early stopping med patience 5.

6. Att se till så att modellen inte blir överanpassad (Overfitted), dvs att den inte lär sig träningsdatan för väl. Då kommer den inte bli bra på att generalisera på ny data vilket leder till fler felaktiga prediktioner.

7. Det är ett sätt för att motverka överanpassning. Detta sker genom att att vissa noder "droppas" vilket medför att modellen inte kan bli beroende av vissa noder vilket tvingar den till att göra alla noder mer "självständiga".

8. Earlystopping används för att sluta träna modellen när den inte längre presterar bättre. Vi anger ett värde för "patience" och när modellen presterat sämre x antal ggr i rad så stannar modellen. Den minns det bästa parametrara från innan den blev sämre.

9. CNN, Convolutional Neural Network.

10. Först identifierar den "low-level features" så som enklare former och färger.
Därefter kombineras dessa för att skapa "high-level features" exempelvis ögon, mun m.m
-För att en modell skall klassas som CNN krävs att minst ett av de dolda lagren är ett Convolutinal layer

11. Den första raden sparar den tränade modellen i en fil med namnet "model_file.keras".
Den andra raden laddar in den sparade modellen från filen och lagrar den i variabeln my_model, så att modellen kan användas igen utan att behöva tränas om.

12. CPU: Datorns processor. Styr och utför allmänna beräkningar.
    <br>GPU: Grafikkortets processor. Gör många beräkningar samtidigt. Lämpar sig därför väl för bilder och ML.

<h3>Självutvärdering</h3>

1. Vad har varit roligast i kunskapskontrollen?


2. Vilket betyg anser du att du ska ha och varför?

3. Vad har varit mest utmanande i arbetet och hur har du hanterat det?